
# Machine learning tutorial
# MLGWSC-1 kickoff meeting
### 12. 10. 2021, Zoom

## What is ML?
**Tom M. Mitchell**: *A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E.*

## Basic formulation of ML problem
We need to approximate function $f^\star : \mathbb{R}^n \rightarrow \mathbb{R}^m,~ f^\star\left(\mathbf{x}\right) = \mathbf{y}$.

Let us define a *model* $f : \mathbb{R}^n \times \mathbb{R}^k \rightarrow \mathbb{R}^m,~ f\left(\mathbf{x}, \theta\right) = \mathbf{y}$; typically: $\mathbf{x}$ = inputs, $\mathbf{y}$ = outputs, $\theta$ = weights.

If the model is suitable to the problem, then a set of weights $\hat{\theta}$ exists such that $f\left(\mathbf{x}, \hat{\theta}\right) \approx f^\star\left(\mathbf{x}\right)$.

### How to find the right weights?

Two more necessary ingredients:

Dataset: $\mathbf{X}, \mathbf{Y}, \forall i: \mathbf{Y}_i = f^\star\left(\mathbf{X}_i\right) + \mathrm{noise}$, $\mathbf{Y}$ are called labels

Loss/cost/error function: $\mathcal{C}\left(\mathbf{Y}, f\left(\mathbf{X}, \theta\right)\right)$ to measure deviation of the model from the labels (e.g. mean squared error)

Training the model: $\hat{\theta} = \mathrm{argmin}_\theta\{\mathcal{C}\left(\mathbf{Y}, f\left(\mathbf{X}, \theta\right)\right)\} \rightarrow \boxed{f\left(\mathbf{x}, \hat{\theta}\right)}$

## Example: Polynomial regression

We'll attempt to approximate the polynomial $f^\star\left(x\right) = 2x - 10x^5 + 15x^{10}$ (*true model*) using polynomial regression: minimization of the mean squared error $\mathcal{C}_{MSE}\left(\mathbf{Y}, f\left(\mathbf{X}, \theta\right)\right) = \frac{1}{N}\sum_{i=0}^{N-1}\left(\mathbf{Y}_i - f\left(\mathbf{X}_i, \theta\right)\right)^2$, and the weights are the coefficients of the polynomial.

If the $\mathbf{X}$ inputs are non-degenerate (no two are the same) and there are more than the polynomial degree, there is a single minimum and it is easy to compute.

We'll sample some values of $f^\star$ with a little white Gaussian noise as a **training dataset**:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng()

### Create the true model and some values to plot
poly = np.polynomial.Polynomial([0., 2., 0., 0., 0., -10., 0., 0., 0., 0., 15.])
plot_X = np.linspace(0., 1., 10**3)
plot_Y = poly(plot_X)
### Create the plot limits
xlim = (0., 1.)
ylim = (min(plot_Y), max(plot_Y))
extend_factor = .1
xlim = (xlim[0]-extend_factor*(xlim[1]-xlim[0]), xlim[1]+extend_factor*(xlim[1]-xlim[0]))
ylim = (ylim[0]-extend_factor*(ylim[1]-ylim[0]), ylim[1]+extend_factor*(ylim[1]-ylim[0]))

In [ ]:
### Generate training data
# tr_X = rng.uniform(0., 1., 11)
tr_X = np.linspace(0., 1., 11)
tr_Y = poly(tr_X) + rng.normal(0., .3, len(tr_X))

### Plot the true model and the training data
plt.figure(figsize=(10., 5.))
plt.xlim(xlim)
plt.ylim(ylim)
plt.plot(plot_X, plot_Y, label='true model')
plt.scatter(tr_X, tr_Y, label='training data')
plt.legend(loc='best')
plt.xlabel('x')
plt.ylabel('y')

Now, let's train some polynomials. Change the line that starts with `deg` to set the orders of fitting polynomials. One might expect that since we have the 11 points required to fit a 10-degree polynomial and the data was generated by a 10-degree polynomial, that would again reproduce the true model the best.

In [ ]:
### Fit polynomials
degs = [1, 3, 5, 10]
fit_polys = [poly.fit(tr_X, tr_Y, deg) for deg in degs]

### Plot the fitted polynomials
plt.figure(figsize=(15., 10.))
plt.xlim(xlim)
plt.ylim(ylim)
for deg, fit_poly in zip(degs, fit_polys):
    plt.plot(plot_X, fit_poly(plot_X), label='%i'%deg)
plt.scatter(tr_X, tr_Y, label='training data')
plt.plot(plot_X, plot_Y, label='true model')
plt.legend(loc='best')
plt.xlabel('x')
plt.ylabel('y')

Generate some **test data** and evaluate the training (*in-sample*) and test (*out-of-sample*) losses of a set of polynomials:

In [ ]:
### Fit polynomials
degs = range(11)
fit_polys = [poly.fit(tr_X, tr_Y, deg) for deg in degs]

### Generate test data, same distribution as training data
te_X = rng.uniform(0., 1., 100)
te_Y = poly(te_X) + rng.normal(0., .3, len(te_X))

### Compute the loss values
losses = []
for deg, fit_poly in zip(degs, fit_polys):
    new_losses = []
    for X, Y in ((tr_X, tr_Y), (te_X, te_Y)):
        model_Y = fit_poly(X)
        new_losses.append(np.mean((model_Y - Y)**2))
    losses.append(new_losses)
losses = np.array(losses)

### Plot the loss values
plt.figure(figsize=(10., 5.))
plt.plot(degs, losses[:, 0], '.-', label='training')
plt.plot(degs, losses[:, 1], '.-', label='test')
plt.semilogy()
plt.ylim((1.e-2, 1.e1))
plt.legend(loc='best')
plt.xlabel('polynomial degree')
plt.ylabel('loss')

In fact, the degree-10 polynomial is not a good fit. The plot above demonstrates that with a limited number of noisy data points, too high complexity model gives the best training loss (so it's a very good fit to that particular set of data), but has trouble generalizing to new data from the same underlying distribution. This phenomenon is called **overfitting**. It is the difference between fitting and predicting.

One must therefore, when working with more complex data and models, be careful about model complexity - increasing it may lead to a better fit on training data and at the same time, to worse predictions on unseen data. Even if a model is perfectly suited to a given underlying true model, in the presence of limited and/or noisy data, a less complex model may reproduce the true model better. The only way to prevent this issue is to keep an eye on the out-of-sample loss. This is achieved by a dataset split.

In some cases, a dataset is supplied already split - in other cases, one must split the data themselves. In a lot of cases where the optimization is iterative, one must have a third separate dataset, called the **validation set**. Upon each iteration of the optimization algorithm over the training set, the loss on the validation set is evaluated to make sure that it is decreasing, and the training is typically interrupted once this is no longer the case. This in turn might also lead to slight overfitting on the validation dataset, which is why we still need the test set despite already having computed an out-of-sample validation loss.

## Gradient descent algorithms

Polynomial regression was a simple example, which is easy to solve - the loss optimization has a single global minimum (under weak assumptions). With more complex ML models, this is no longer the case. Usually, gradient descent-based algorithms are used to find the minimum of $E\left(\theta\right) = \mathcal{C}\left(\mathbf{Y}, f\left(\mathbf{X}, \theta\right)\right)$. These algorithms require an "initial position" $\theta_0$ and attempt to find the solution iteratively.

Basic gradient descent:
$$\mathbf{v}_t = \eta\nabla_\theta E\left(\theta_t\right),~\theta_{t+1} = \theta_t - \mathbf{v}_t$$

Let's try on a simple parabolic surface $E\left(\theta\right) = \theta_0^2 + \theta_1^2$. Feel free to play around with the value of `eta` to see why some choices of learning rate are unsuitable.

In [ ]:
### Parabolic surface and its gradient
def surface(theta):
    return np.sum(theta**2, axis=0)

def gradient(theta):
    return 2*theta

### Generate contour plot data
X = np.linspace(-5., 5., 1000)
Y = np.linspace(-5., 5., 1000)
Z = surface(np.stack((np.tile(np.expand_dims(X, 1), (1, len(Y))), np.tile(np.expand_dims(Y, 0), (len(X), 1))), axis=0)).T

### Initial position and GD parameters
theta = [np.array((-4., -2.))]
epochs = range(21)
eta = 0.5

### GD loop
for e in epochs[1:]:
    v = eta*gradient(theta[-1])
    theta.append(theta[-1]-v)
theta = np.stack(theta, axis=1)

### Plot results
plt.figure(figsize=(7., 7.))
plt.contour(X, Y, Z)
plt.plot(theta[0], theta[1], '.-', linewidth=1., markersize=10.)
plt.axis('square')
plt.xlim((min(X), max(X)))
plt.ylim((min(Y), max(Y)))

Usually, the landscape which we need our optimizer to navigate will be much more rugged with local minima and saddle points - and much higher-dimensional, so impossible to visualize! This makes it much harder to determine a suitable learning rate. However, smart adaptive algorithms exist which keep slowly decaying running averages of the gradients and other properties, which allows them to locally estimate the Hessian matrix and effectively adapt the learning rate to its ideal value. We won't learn about these in this tutorial - to try them out, simply replace the `torch.optim.SGD` by, e.g., `torch.optim.Adam`.

To be able to work with complex data, we need complex models - so we need the datasets to be large. Computation of the gradient of $E\left(\theta\right)$ over the full training dataset is then computationally very difficult, let alone doing several tens or hundreds of optimization steps, so we need to apply one more approximation to our method.

We assume that evaluating the loss gradient over a smaller subset of the training dataset is representative of the full computation. We split the training dataset into **batches** and do a single step of the gradient descent method on each of these batches. One iteration over all the batches is called a **training epoch**. This method is called **stochastic gradient descent**. The same principle is also used with the adaptive optimizers mentioned above.

## Neural Networks

Neural networks are inspired by the human neural system. The basic element is a very simple **artificial neuron**, which merely takes a linear combination of its input values, with an additional constant term, and applies a non-linear function:
$$y(\mathbf{x}) = \sigma\left(\sum_{i} w_ix_i + b\right).$$

The $w_i$ values are usually called **weights** and $b$ is the **bias**. The function $\sigma$ is called the **activation function** and there are many different choices. Some of the most popular are, for example:
$$\mathrm{Sigmoid}\left(x\right) = \frac{1}{1+e^{-x}},~\mathrm{ReLU}\left(x\right) = \mathrm{max}\left(0, x\right)~.$$

For classification problems, the *Softmax* is very popular, because it maps $\mathbb{R}^N$ to a set of positive numbers that sum up to one. This is often used as a final activation to produce approximate probabilities as a model output (in other places in the network, however, others are typically used).
$$\mathrm{Softmax}\left(\mathbf{x}\right)_i = \frac{\exp\left(x_i\right)}{\sum_j \exp\left(x_j\right)}$$

In [ ]:
### Plot the Softmax and ReLU activation functions
X = np.linspace(-10., 10., 10000)
fig, axes = plt.subplots(ncols=2, figsize=(10., 5.))

Y = 1./(1.+np.exp(-X))
axes[0].plot(X, Y, label='Sigmoid')
axes[0].legend(loc='best')

Y = np.amax(np.stack((X, np.zeros_like(X)), axis=0), axis=0)
axes[1].plot(X, Y, label='ReLU')
axes[1].legend(loc='best')

We can order multiple independent neurons into a **layer**, so that:
$$y_i\left(\mathbf{x}\right) = \sigma\left(\sum_j w_{ij}x_j + b_i\right).$$

Let us apply this to a real-life classification problem. We will use the MNIST dataset, which contains 60 000 annotated hand-written digits as single-channel $28\times 28$ images and 10 000 more for testing.

In [ ]:
import torch, torchvision
fname = 'MNIST'

### Download and initialize datasets
TrainDS_orig = torchvision.datasets.MNIST(fname, train=True, download=True)
TestDS_orig = torchvision.datasets.MNIST(fname, train=False)

Plot a few examples of the data:

In [ ]:
### Plot examples
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
for axes_row in axes:
    for ax in axes_row:
        test_index = rng.integers(0, len(TestDS_orig))
        image, orig_label = TestDS_orig[test_index]
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i' % orig_label)

Let's try to use a simple neuron layer to classify the digits in the MNIST dataset. We first need to create a function that will transform an integer label to a set of probabilities, e.g., $1 \to (0., 1., 0., ..., 0.)$: $i$-th element represents the probability that the sample contains the digit $i$. We reinitialize the datasets using this transform as well as one which turns our input images into $28\times 28$ PyTorch tensors.

Then we define the model itself as a sequence of functions. In the PyTorch vernacular, the first is `torch.nn.Flatten`; it rearranges the input $28\times 28$ tensor into a vector of length $28\cdot 28 = 784$. The next `torch.nn.Linear` is the linear combination performed by the neurons: this outputs 10 numbers, each of which is a linear combination of the 784 inputs with independent coefficients as well as constant biases, all of which are trainable weights. Finally, the `torch.nn.Softmax` performs the Softmax operation as described a few blocks above. We will use the SGD method with learning rate 0.1 and batch size 32 and as a loss function, we will use the binary cross entropy loss, which is the canonical choice for classification problems.

In [ ]:
### Define the label transform from an integer to a set of probabilities
def target_transform(inlabel):
    newlabel = torch.zeros(10)
    newlabel[inlabel] = 1.
    return newlabel

### Reinitialize datasets with the transforms
TrainDS = torchvision.datasets.MNIST(fname, train=True, download=True,
            target_transform=target_transform, transform=torchvision.transforms.ToTensor())
TestDS = torchvision.datasets.MNIST(fname, train=False,
            target_transform=target_transform, transform=torchvision.transforms.ToTensor())

### Initialize DataLoaders as PyTorch convenience
TrainDL = torch.utils.data.DataLoader(TrainDS, shuffle=True, batch_size=32)
TestDL = torch.utils.data.DataLoader(TestDS, batch_size=1000)

### Choose device: 'cuda' or 'cpu'
device = 'cuda:0'

### Define the dense neuron layer
Network = torch.nn.Sequential(
    torch.nn.Flatten(),            # 28x28 -> 784
    torch.nn.Linear(784, 10),    # 784 -> 10
    torch.nn.Softmax(dim=1)
)
Network.to(device=device)

### Get information about model
totpars = 0
for par in Network.parameters():
    newpars = 1
    for num in par.shape:
        newpars *= num
    totpars += newpars
print(Network)
print('%i trainable parameters' % totpars)

### Initialize loss function and optimizer
crit = torch.nn.BCELoss()
opt = torch.optim.SGD(Network.parameters(), lr=0.1)

Most of us don't really have an idea what a good value for the binary cross entropy is, so for reference, let's calculate what the loss will be if our model simply estimates for each sample that each digit has an 0.1 probability to be present:

In [ ]:
### Baseline: just say it's anything at probability 1/N, what's the loss?
N = 10
labels = torch.zeros(1, 10, dtype=torch.float32)
labels[0, 3] = 1.
output = torch.full_like(labels, 1./N)
print(crit(output, labels))

Now we have a baseline for the loss value.

We finally get to training the model. The following block performs 10 training epochs, in each of them loops over all the 32-samples-long batches in the training dataset, repeating the following steps: set the gradient values to zero (these are stored as attributes of the relevant tensors), move the batch samples and labels to the proper device (CPU or CUDA), compute the outputs of the model on that particular batch, compute the loss value, compute the gradients using back-propagation (this is the algorithm that computes the gradients on neural networks) and performs the optimization step. At the end of each epoch, the loss values over all the training samples are averaged and printed along the epoch number.

In [ ]:
### Set model in training mode and create the epochs axis
Network.train()
epochs = range(1, 11)

### Train the model
for e in epochs:
    tr_loss = 0.
    samples = 0
    ### Loop over batches
    for inputs, labels in TrainDL:
        opt.zero_grad() # zero gradient values
        inputs = inputs.to(device=device) # move input and label tensors to the device with the model
        labels = labels.to(device=device)
        outputs = Network(inputs) # compute model outputs
        loss = crit(outputs, labels) # compute batch loss
        loss.backward() # back-propagate the gradients
        opt.step() # update the model weights
        tr_loss += loss.clone().cpu().item()*len(inputs) # add the batch loss to the running loss
        samples += len(inputs) # update the number of processed samples
    tr_loss /= samples # compute training loss
    print(e, tr_loss)

Now our model is trained. The in-sample (training) loss has decreased considerably, to much less than the uniform probabilities baseline. We need to make sure, however, that the model can generalize and hasn't merely memorized the training dataset. To achieve that, we evaluate the loss on the test dataset. In that process, let's also determine the **accuracy**: the percentage of test samples where the model has assigned the highest probability to the correct class.

In [ ]:
### Set model in evaluation mode
Network.eval()

### Compute the test loss
with torch.no_grad():
    te_loss = 0.
    samples = 0
    accuracy = 0
    ### Loop over batches
    for inputs, labels in TestDL:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        outputs = Network(inputs)
        loss = crit(outputs, labels)
        te_loss += loss.clone().cpu().item()*len(inputs)
        accuracy += torch.sum(torch.eq(torch.max(labels, 1)[1], torch.max(outputs, 1)[1]), dtype=int).clone().cpu().item()
        samples += len(inputs)
    te_loss /= samples
    accuracy /= samples
    print('Test loss: %f, accuracy: %f' % (te_loss, accuracy))

The test loss is close to the training loss. This means that we haven't run into issues with overfitting. The accuracy also seems pretty good - should be around 92%! Let us now see some of the results - out of the following two blocks, the first will show 12 random test examples of digits with their true as well as predicted classes and the confidence level the model has about its prediction, and the second will do the same but only draw from those the model has classified incorrectly.

In [ ]:
### Draw some random images from the test dataset and compare the true labels to the network outputs
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        ### Draw the images
        test_index = rng.integers(0, len(TestDS))
        sample, label = TestDS[test_index]
        image, orig_label = TestDS_orig[test_index]
        ### Compute the predictions
        with torch.no_grad():
            output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
            certainty, output = torch.max(output[0], 0)
            certainty = certainty.clone().cpu().item()
            output = output.clone().cpu().item()
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

In [ ]:
### Draw random images from the test dataset and select and show some which have been incorrectly classified
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        while True:
            ### Draw the images
            test_index = rng.integers(0, len(TestDS))
            sample, label = TestDS[test_index]
            image, orig_label = TestDS_orig[test_index]
            ### Compute the predictions
            with torch.no_grad():
                output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
                certainty, output = torch.max(output[0], 0)
                certainty = certainty.clone().cpu().item()
                output = output.clone().cpu().item()
                if output!=orig_label:
                    break
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

## Deep neural networks

This is already a simple neural network. However, to build a **deep neural network**, we need to stack several such layers, such that each subsequent layer takes the output of the previous layer as its input. This method of stacking allows us to build very complex models - while the individual structural elements are very simple, the non-linear elements weaved into the network of linear operations allow the model to approximate any function to an arbitrary degree as long as you're flexible on depth and layer size.

In [ ]:
### Show schematic of a fully connected deep neural network
import matplotlib.image as mpimg
fig, ax = plt.subplots(figsize=(15., 7.))
ax.set_axis_off()
ax.imshow(mpimg.imread('nn.png'))

This network will only have one additional layer to what we've used before. It is again a sequence of PyTorch modules - but there is now an additional layer. The first layer gives 50 outputs with the $\mathrm{ReLU}\left(x\right) = \mathrm{max}\left(0, x\right)$ activation function and the second gives 10 outputs with the $\mathrm{Softmax}$ activation. The rest of the code in this section works the same way as with the previous model.

In [ ]:
### Define a simple two-layer network
Network = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 10),
    torch.nn.Softmax(dim=1)
)
Network.to(device=device)

### Get information about model
totpars = 0
for par in Network.parameters():
    newpars = 1
    for num in par.shape:
        newpars *= num
    totpars += newpars
print(Network)
print('%i trainable parameters' % totpars)

### Initialize loss function and optimizer
crit = torch.nn.BCELoss()
opt = torch.optim.SGD(Network.parameters(), lr=0.1)

In [ ]:
### Set model in training mode and create the epochs axis
Network.train()
epochs = range(1, 11)

### Train the model
for e in epochs:
    tr_loss = 0.
    samples = 0
    ### Loop over batches
    for inputs, labels in TrainDL:
        opt.zero_grad() # zero gradient values
        inputs = inputs.to(device=device) # move input and label tensors to the device with the model
        labels = labels.to(device=device)
        outputs = Network(inputs) # compute model outputs
        loss = crit(outputs, labels) # compute batch loss
        loss.backward() # back-propagate the gradients
        opt.step() # update the model weights
        tr_loss += loss.clone().cpu().item()*len(inputs) # add the batch loss to the running loss
        samples += len(inputs) # update the number of processed samples
    tr_loss /= samples # compute training loss
    print(e, tr_loss)

In [ ]:
### Set model in evaluation mode
Network.eval()

### Compute the test loss
with torch.no_grad():
    te_loss = 0.
    samples = 0
    accuracy = 0
    ### Loop over batches
    for inputs, labels in TestDL:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        outputs = Network(inputs)
        loss = crit(outputs, labels)
        te_loss += loss.clone().cpu().item()*len(inputs)
        accuracy += torch.sum(torch.eq(torch.max(labels, 1)[1], torch.max(outputs, 1)[1]), dtype=int).clone().cpu().item()
        samples += len(inputs)
    te_loss /= samples
    accuracy /= samples
    print('Test loss: %f, accuracy: %f' % (te_loss, accuracy))

This model has about 4 times as many trainable parameters as the previous one, which brings an increased risk of overfitting. However, the test loss indicates that we haven't run into overfitting issues yet.

In [ ]:
### Draw some random images from the test dataset and compare the true labels to the network outputs
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        ### Draw the images
        test_index = rng.integers(0, len(TestDS))
        sample, label = TestDS[test_index]
        image, orig_label = TestDS_orig[test_index]
        ### Compute the predictions
        with torch.no_grad():
            output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
            certainty, output = torch.max(output[0], 0)
            certainty = certainty.clone().cpu().item()
            output = output.clone().cpu().item()
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

In [ ]:
### Draw random images from the test dataset and select and show some which have been incorrectly classified
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        while True:
            ### Draw the images
            test_index = rng.integers(0, len(TestDS))
            sample, label = TestDS[test_index]
            image, orig_label = TestDS_orig[test_index]
            ### Compute the predictions
            with torch.no_grad():
                output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
                certainty, output = torch.max(output[0], 0)
                certainty = certainty.clone().cpu().item()
                output = output.clone().cpu().item()
                if output!=orig_label:
                    break
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

## Convolutional neural networks

A slightly more convoluted (sorry for the bad pun) type of neural networks has been inspired by the animal visual cortex. Convolutional layers are introduced, which slide *filters* over an input image, producing another image - often with a larger number of channels, so these can be hard to visualize. Each pixel in that output image is then basically the product of a smaller fully connected layer (like we used before) applied to a small portion of the input image. These layers are used to build **convolutional neural networks**, which typically consist of a convolutional part and a dense part. The convolutional part is constructed out of alternating convolutional layers and pooling layers (these downsample the image between the convolutional layers), the result is flattened and passed to a dense part, which is typically a deep neural network as shown before.

This structure removes some less crucial connections between neurons and use many weights for multiple connections as opposed to a fully connected network. This allows us to build a network just as suited to some problems as a fully connected network while dramatically reducing the number of free parameters to be optimized. They have proven to be particularly suitable to computer vision problems.

Go ahead and play with the following - it's the same code as before, just a different network design.

In [ ]:
fig, ax = plt.subplots(figsize=(15., 7.))
ax.set_axis_off()
ax.imshow(mpimg.imread('cnn.jpg'))

In [ ]:
### Create a simple convolutional neural network
Network = torch.nn.Sequential(      #  1x28x28
    torch.nn.Conv2d(1, 12, (9, 9)),  #  12x20x20
    torch.nn.MaxPool2d((2, 2)),     #  12x10x10
    torch.nn.ReLU(),
    torch.nn.Conv2d(12, 24, (5, 5)), # 24x 6x 6
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((3, 3)),     # 24x 2x 2
    torch.nn.Flatten(),             #       96
    torch.nn.Linear(96, 16),        #       16
    torch.nn.ReLU(),
    torch.nn.Linear(16, 10),        #       10
    torch.nn.Softmax(dim=1)
)
Network.to(device=device)

### Get information about model
totpars = 0
for par in Network.parameters():
    newpars = 1
    for num in par.shape:
        newpars *= num
    totpars += newpars
print(Network)
print('%i trainable parameters' % totpars)

### Initialize loss function and optimizer
opt = torch.optim.SGD(Network.parameters(), lr=.1)

In [ ]:
### Set model in training mode and create the epochs axis
Network.train()
epochs = range(1, 11)

### Train the model
for e in epochs:
    tr_loss = 0.
    samples = 0
    ### Loop over batches
    for inputs, labels in TrainDL:
        opt.zero_grad() # zero gradient values
        inputs = inputs.to(device=device) # move input and label tensors to the device with the model
        labels = labels.to(device=device)
        outputs = Network(inputs) # compute model outputs
        loss = crit(outputs, labels) # compute batch loss
        loss.backward() # back-propagate the gradients
        opt.step() # update the model weights
        tr_loss += loss.clone().cpu().item()*len(inputs) # add the batch loss to the running loss
        samples += len(inputs) # update the number of processed samples
    tr_loss /= samples # compute training loss
    print(e, tr_loss)

In [ ]:
### Set model in evaluation mode
Network.eval()

### Compute the test loss
with torch.no_grad():
    te_loss = 0.
    samples = 0
    accuracy = 0
    ### Loop over batches
    for inputs, labels in TestDL:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        outputs = Network(inputs)
        loss = crit(outputs, labels)
        te_loss += loss.clone().cpu().item()*len(inputs)
        accuracy += torch.sum(torch.eq(torch.max(labels, 1)[1], torch.max(outputs, 1)[1]), dtype=int).clone().cpu().item()
        samples += len(inputs)
    te_loss /= samples
    accuracy /= samples
    print('Test loss: %f, accuracy: %f' % (te_loss, accuracy))

In [ ]:
### Draw some random images from the test dataset and compare the true labels to the network outputs
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        ### Draw the images
        test_index = rng.integers(0, len(TestDS))
        sample, label = TestDS[test_index]
        image, orig_label = TestDS_orig[test_index]
        ### Compute the predictions
        with torch.no_grad():
            output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
            certainty, output = torch.max(output[0], 0)
            certainty = certainty.clone().cpu().item()
            output = output.clone().cpu().item()
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

In [ ]:
### Draw random images from the test dataset and select and show some which have been incorrectly classified
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(15., 6.))
### Loop over subplots
for axes_row in axes:
    for ax in axes_row:
        while True:
            ### Draw the images
            test_index = rng.integers(0, len(TestDS))
            sample, label = TestDS[test_index]
            image, orig_label = TestDS_orig[test_index]
            ### Compute the predictions
            with torch.no_grad():
                output = Network(torch.unsqueeze(sample, dim=0).to(device=device))
                certainty, output = torch.max(output[0], 0)
                certainty = certainty.clone().cpu().item()
                output = output.clone().cpu().item()
                if output!=orig_label:
                    break
        ### Show image
        ax.set_axis_off()
        ax.imshow(image)
        ax.set_title('True: %i, predicted: %i\nat %f' % (orig_label, output, certainty))

Congratulations! You have managed to get to the point where your network mostly misclassifies only those digits, which are badly written and are outliers in the dataset. We've only scratched the surface of what you can do with neural networks, but you've learned the most elementary things you need to know. Go ahead and play with this notebook - with the networks, with the training parameters, etc. Good luck!